In [1]:
import numpy as np
import pandas as pd

In [2]:
our_count = pd.read_csv("../ressources/result_matrix/matrix_comptage.csv")
our_count = our_count.rename(columns={"Unnamed: 0": "Name"})
our_count.Name = our_count.Name.replace({'gene-':''}, regex=True)
our_count.head()

,Name,IP1,IP2,IP3,control4,control5,control6,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,SAOUHSC_00001,20822,17144,13268,14204,19660,4006,13319.548363,0.179625,0.285666,0.628795,0.529483,0.613923
1,SAOUHSC_00002,16122,12278,11242,12226,15120,2134,9799.576106,0.343153,0.174728,1.963924,0.049539,0.084813
2,SAOUHSC_00003,1631,1215,881,1382,1024,126,854.819460,0.555984,0.345394,1.609708,0.107462,0.164831
3,SAOUHSC_00004,14058,11486,9997,7628,6737,846,7036.337157,1.221752,0.268818,4.544910,0.000005,0.000025
4,SAOUHSC_00005,29702,26506,24292,12095,12108,3508,16187.904117,1.187337,0.341595,3.475865,0.000509,0.001475


In [3]:
ground_truth = pd.read_csv("../ressources/result_matrix/matrix_article.csv")
ground_truth = ground_truth.rename(columns={"ctrl4":"control4", "ctrl5":"control5", "ctrl6":"control6"})
ground_truth.head()

,Id,Name,Name.NCBI,entrez,product,control4,control5,control6,IP1,IP2,IP3,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,gene0,SAOUHSC_00001,ABD29192.1,3919798.0,chromosomal replication initiator protein DnaA,14094.0,19538.0,3983.0,20639.0,17017.0,13167.0,13230.327020,0.185266,0.285612,0.648663,5.165565e-01,6.047491e-01
1,gene1,SAOUHSC_00002,ABD29193.1,3919799.0,DNA polymerase III beta subunit,12111.0,15005.0,2112.0,15939.0,12157.0,11127.0,9703.317278,0.350954,0.170702,2.055945,3.978783e-02,6.960326e-02
2,gene10,SAOUHSC_00012,ABD29202.1,3919184.0,conserved hypothetical protein,1156.0,2751.0,299.0,208.0,175.0,171.0,675.548617,-2.948202,0.325776,-9.049786,1.430000e-19,3.840000e-18
3,gene100,SAOUHSC_00107,ABD29288.1,3919816.0,conserved hypothetical protein,1832.0,2369.0,530.0,2228.0,1813.0,1587.0,1579.419184,-0.021133,0.293272,-0.072060,9.425545e-01,9.565389e-01
4,gene1000,SAOUHSC_01039,ABD30159.1,3919887.0,conserved hypothetical protein,5621.0,6283.0,260.0,3481.0,2167.0,2103.0,2489.561063,-0.427325,0.483853,-0.883171,3.771438e-01,4.707415e-01


In [4]:
df_coincidences = pd.merge(our_count, ground_truth, how='inner', on = ['Name'])
df_coincidences.head(5)

,Name,IP1_x,IP2_x,IP3_x,control4_x,control5_x,control6_x,baseMean_x,log2FoldChange_x,lfcSE_x,...,control6_y,IP1_y,IP2_y,IP3_y,baseMean_y,log2FoldChange_y,lfcSE_y,stat_y,pvalue_y,padj_y
0,SAOUHSC_00001,20822,17144,13268,14204,19660,4006,13319.548363,0.179625,0.285666,...,3983.0,20639.0,17017.0,13167.0,13230.327020,0.185266,0.285612,0.648663,0.516556,0.604749
1,SAOUHSC_00002,16122,12278,11242,12226,15120,2134,9799.576106,0.343153,0.174728,...,2112.0,15939.0,12157.0,11127.0,9703.317278,0.350954,0.170702,2.055945,0.039788,0.069603
2,SAOUHSC_00003,1631,1215,881,1382,1024,126,854.819460,0.555984,0.345394,...,126.0,1624.0,1211.0,875.0,850.895748,0.567755,0.338535,1.677094,0.093524,0.146805
3,SAOUHSC_00004,14058,11486,9997,7628,6737,846,7036.337157,1.221752,0.268818,...,837.0,13949.0,11407.0,9898.0,6983.207358,1.233104,0.262615,4.695476,0.000003,0.000012
4,SAOUHSC_00005,29702,26506,24292,12095,12108,3508,16187.904117,1.187337,0.341595,...,3485.0,29476.0,26337.0,24130.0,16102.740090,1.192714,0.340332,3.504562,0.000457,0.001317


In [5]:
df_comp = df_coincidences.loc[:,["Name", "IP1_x", "IP1_y", "IP2_x", "IP2_y", "IP3_x", "IP3_y", "control4_x", "control4_y", "control5_x", "control5_y", "control6_x", "control6_y"]]

list_errors = ["IP1", "IP2", "IP3", "control4", "control5", "control6"]

for item in list_errors:
  df_comp["abs_error_" + item] = np.abs(df_comp[item + "_x"]-df_comp[item + "_y"])
  df_comp["rela_error_" + item] = df_comp["abs_error_" + item]/df_comp[item + "_y"]

df_comp
df_comp.isna().sum()
df_comp = df_comp.dropna()

with pd.option_context("mode.use_inf_as_na", True):
  df_comp.dropna(inplace=True)

#Creating a matrix only with the errors
df_errors = df_comp.sort_values(by=["rela_error_IP1"])
df_errors.head(10)

/tmp/ipykernel_18612/1083052451.py:13: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):


,Name,IP1_x,IP1_y,IP2_x,IP2_y,IP3_x,IP3_y,control4_x,control4_y,control5_x,...,abs_error_IP2,rela_error_IP2,abs_error_IP3,rela_error_IP3,abs_error_control4,rela_error_control4,abs_error_control5,rela_error_control5,abs_error_control6,rela_error_control6
2717,SAOUHSC_02791,171,171.0,109,111.0,133,134.0,262,260.0,447,...,2.0,0.018018,1.0,0.007463,2.0,0.007692,3.0,0.006757,1.0,0.025000
1762,SAOUHSC_01844,7,7.0,8,7.0,7,6.0,3,3.0,5,...,1.0,0.142857,1.0,0.166667,0.0,0.000000,0.0,0.000000,0.0,0.000000
1799,SAOUHSC_01883,15,15.0,8,8.0,25,25.0,11,12.0,36,...,0.0,0.000000,0.0,0.000000,1.0,0.083333,0.0,0.000000,0.0,0.000000
2779,SAOUHSC_02857,52,52.0,45,45.0,68,68.0,748,745.0,824,...,0.0,0.000000,0.0,0.000000,3.0,0.004027,3.0,0.003654,2.0,0.010811
1245,SAOUHSC_01301,64,64.0,43,43.0,106,105.0,284,283.0,435,...,0.0,0.000000,1.0,0.009524,1.0,0.003534,0.0,0.000000,0.0,0.000000
1241,SAOUHSC_01297,22,22.0,9,9.0,18,18.0,86,85.0,166,...,0.0,0.000000,0.0,0.000000,1.0,0.011765,1.0,0.006061,1.0,0.125000
714,SAOUHSC_00745,15,15.0,4,4.0,11,11.0,4,4.0,7,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.166667,0.0,0.000000
1238,SAOUHSC_01294,14,14.0,6,4.0,12,12.0,9,9.0,11,...,2.0,0.500000,0.0,0.000000,0.0,0.000000,1.0,0.100000,0.0,0.000000
1235,SAOUHSC_01291,17,17.0,11,11.0,12,12.0,28,26.0,69,...,0.0,0.000000,0.0,0.000000,2.0,0.076923,0.0,0.000000,0.0,0.000000
1838,SAOUHSC_01924,47,47.0,18,17.0,19,19.0,21,21.0,35,...,1.0,0.058824,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [6]:
df_errors.tail(10)

,Name,IP1_x,IP1_y,IP2_x,IP2_y,IP3_x,IP3_y,control4_x,control4_y,control5_x,...,abs_error_IP2,rela_error_IP2,abs_error_IP3,rela_error_IP3,abs_error_control4,rela_error_control4,abs_error_control5,rela_error_control5,abs_error_control6,rela_error_control6
1934,SAOUHSC_T00024,159,16.0,105,21.0,101,19.0,58,6.0,20,...,84.0,4.000000,82.0,4.315789,52.0,8.666667,15.0,3.000000,59.0,59.000000
1936,SAOUHSC_T00036,1014,101.0,1225,136.0,525,66.0,453,53.0,167,...,1089.0,8.007353,459.0,6.954545,400.0,7.547170,157.0,15.700000,424.0,30.285714
2955,SAOUHSC_03042,356,33.0,221,34.0,291,55.0,215,27.0,301,...,187.0,5.500000,236.0,4.290909,188.0,6.962963,256.0,5.688889,23.0,2.555556
1915,SAOUHSC_T00016,1764,139.0,1251,120.0,1058,97.0,976,77.0,169,...,1131.0,9.425000,961.0,9.907216,899.0,11.675325,158.0,14.363636,300.0,12.500000
1919,SAOUHSC_T00054,2157,118.0,2213,98.0,1601,78.0,712,50.0,705,...,2115.0,21.581633,1523.0,19.525641,662.0,13.240000,696.0,77.333333,272.0,27.200000
1937,SAOUHSC_T00055,1081,50.0,506,25.0,537,28.0,433,15.0,178,...,481.0,19.240000,509.0,18.178571,418.0,27.866667,170.0,21.250000,249.0,12.450000
1722,SAOUHSC_01804,259,9.0,178,7.0,198,11.0,142,14.0,96,...,171.0,24.428571,187.0,17.000000,128.0,9.142857,90.0,15.000000,8.0,8.000000
473,SAOUHSC_T00030,482,11.0,339,24.0,277,4.0,387,19.0,74,...,315.0,13.125000,273.0,68.250000,368.0,19.368421,73.0,73.000000,76.0,76.000000
995,SAOUHSC_A00992,1467,29.0,1403,27.0,913,15.0,1212,22.0,1353,...,1376.0,50.962963,898.0,59.866667,1190.0,54.090909,1333.0,66.650000,380.0,42.222222
1940,SAOUHSC_R0008,6363,4.0,6897,9.0,2119,1.0,5389,2.0,3995,...,6888.0,765.333333,2118.0,2118.000000,5387.0,2693.500000,3993.0,1996.500000,8423.0,1403.833333


In [7]:
df_comp

,Name,IP1_x,IP1_y,IP2_x,IP2_y,IP3_x,IP3_y,control4_x,control4_y,control5_x,...,abs_error_IP2,rela_error_IP2,abs_error_IP3,rela_error_IP3,abs_error_control4,rela_error_control4,abs_error_control5,rela_error_control5,abs_error_control6,rela_error_control6
0,SAOUHSC_00001,20822,20639.0,17144,17017.0,13268,13167.0,14204,14094.0,19660,...,127.0,0.007463,101.0,0.007671,110.0,0.007805,122.0,0.006244,23.0,0.005775
1,SAOUHSC_00002,16122,15939.0,12278,12157.0,11242,11127.0,12226,12111.0,15120,...,121.0,0.009953,115.0,0.010335,115.0,0.009495,115.0,0.007664,22.0,0.010417
2,SAOUHSC_00003,1631,1624.0,1215,1211.0,881,875.0,1382,1372.0,1024,...,4.0,0.003303,6.0,0.006857,10.0,0.007289,5.0,0.004907,0.0,0.000000
3,SAOUHSC_00004,14058,13949.0,11486,11407.0,9997,9898.0,7628,7568.0,6737,...,79.0,0.006926,99.0,0.010002,60.0,0.007928,53.0,0.007929,9.0,0.010753
4,SAOUHSC_00005,29702,29476.0,26506,26337.0,24292,24130.0,12095,12024.0,12108,...,169.0,0.006417,162.0,0.006714,71.0,0.005905,73.0,0.006066,23.0,0.006600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,SAOUHSC_03051,9582,9495.0,7431,7362.0,9008,8945.0,3075,3055.0,4508,...,69.0,0.009372,63.0,0.007043,20.0,0.006547,25.0,0.005577,8.0,0.011730
2963,SAOUHSC_03052,21831,21674.0,18548,18425.0,20302,20160.0,6515,6470.0,9165,...,123.0,0.006676,142.0,0.007044,45.0,0.006955,44.0,0.004824,3.0,0.002004
2964,SAOUHSC_03053,13934,13827.0,11393,11309.0,13413,13327.0,3674,3654.0,4486,...,84.0,0.007428,86.0,0.006453,20.0,0.005473,21.0,0.004703,5.0,0.005841
2965,SAOUHSC_03054,1878,1855.0,1506,1488.0,1607,1588.0,506,499.0,451,...,18.0,0.012097,19.0,0.011965,7.0,0.014028,1.0,0.002222,0.0,0.000000


In [8]:
df_comp[df_comp.abs_error_IP1 == df_comp.abs_error_IP1.max()]

,Name,IP1_x,IP1_y,IP2_x,IP2_y,IP3_x,IP3_y,control4_x,control4_y,control5_x,...,abs_error_IP2,rela_error_IP2,abs_error_IP3,rela_error_IP3,abs_error_control4,rela_error_control4,abs_error_control5,rela_error_control5,abs_error_control6,rela_error_control6
30,SAOUHSC_00031,28787,8065.0,18195,5171.0,29951,8278.0,2106,661.0,2358,...,13024.0,2.518662,21673.0,2.618144,1445.0,2.186082,1583.0,2.042581,232.0,2.729412


In [9]:
df_onlyerrors = df_errors.iloc[:,[0,13,14,15,16,17,18,19,20,21,22,23,24]]
df_onlyerrors

,Name,abs_error_IP1,rela_error_IP1,abs_error_IP2,rela_error_IP2,abs_error_IP3,rela_error_IP3,abs_error_control4,rela_error_control4,abs_error_control5,rela_error_control5,abs_error_control6,rela_error_control6
2717,SAOUHSC_02791,0.0,0.000000,2.0,0.018018,1.0,0.007463,2.0,0.007692,3.0,0.006757,1.0,0.025000
1762,SAOUHSC_01844,0.0,0.000000,1.0,0.142857,1.0,0.166667,0.0,0.000000,0.0,0.000000,0.0,0.000000
1799,SAOUHSC_01883,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.083333,0.0,0.000000,0.0,0.000000
2779,SAOUHSC_02857,0.0,0.000000,0.0,0.000000,0.0,0.000000,3.0,0.004027,3.0,0.003654,2.0,0.010811
1245,SAOUHSC_01301,0.0,0.000000,0.0,0.000000,1.0,0.009524,1.0,0.003534,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937,SAOUHSC_T00055,1031.0,20.620000,481.0,19.240000,509.0,18.178571,418.0,27.866667,170.0,21.250000,249.0,12.450000
1722,SAOUHSC_01804,250.0,27.777778,171.0,24.428571,187.0,17.000000,128.0,9.142857,90.0,15.000000,8.0,8.000000
473,SAOUHSC_T00030,471.0,42.818182,315.0,13.125000,273.0,68.250000,368.0,19.368421,73.0,73.000000,76.0,76.000000
995,SAOUHSC_A00992,1438.0,49.586207,1376.0,50.962963,898.0,59.866667,1190.0,54.090909,1333.0,66.650000,380.0,42.222222


In [25]:
seuil = 0.05
print(df_onlyerrors.shape)
df_smallerrors = df_onlyerrors[(df_onlyerrors.rela_error_IP1 <= seuil) & 
                               (df_onlyerrors.rela_error_IP2 <= seuil) &
                               (df_onlyerrors.rela_error_IP3 <= seuil) &
                               (df_onlyerrors.rela_error_control4 <= seuil) &
                               (df_onlyerrors.rela_error_control5 <= seuil) &
                               (df_onlyerrors.rela_error_control6 <= seuil)]
print(df_smallerrors.shape)

print(f'{df_onlyerrors.shape[0]-df_smallerrors.shape[0]} gènes diffèrent significativement (> {seuil*100}%) en ce qui concerne le comptage.\n \
      c.a.d le {'{:.2f}'.format(df_smallerrors.shape[0] / df_onlyerrors.shape[0] * 100)}% des resultats de comptage obtenus de notre part son objectivement similaires à celles obtenues par les auteurs.')

(2660, 13)
(2382, 13)
278 gènes diffèrent significativement (> 5.0%) en ce qui concerne le comptage.
       c.a.d le 89.55% des resultats de comptage obtenus de notre part son objectivement similaires à celles obtenues par les auteurs.


In [26]:
df_smallerrors.describe()

,abs_error_IP1,rela_error_IP1,abs_error_IP2,rela_error_IP2,abs_error_IP3,rela_error_IP3,abs_error_control4,rela_error_control4,abs_error_control5,rela_error_control5,abs_error_control6,rela_error_control6
count,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000,2382.000000
mean,87.963476,0.012340,64.227540,0.009515,59.584383,0.010260,69.781696,0.007982,69.971872,0.006216,17.241814,0.006372
std,262.189133,0.007456,215.521813,0.006147,181.858168,0.006449,192.200483,0.005080,226.181430,0.004710,100.243485,0.007117
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,0.007742,7.000000,0.006174,7.000000,0.006441,6.000000,0.005449,6.000000,0.004149,1.000000,0.001451
50%,28.000000,0.010055,17.000000,0.007979,16.000000,0.008526,19.000000,0.006893,20.000000,0.005362,3.000000,0.004908
75%,66.000000,0.014659,45.000000,0.010961,43.000000,0.012195,58.000000,0.009140,55.750000,0.007019,10.000000,0.008333
max,5429.000000,0.048780,5444.000000,0.050000,3694.000000,0.050000,4799.000000,0.047974,5886.000000,0.049724,4237.000000,0.050000
